# 範例 15：準備 Fine-Tuning 資料集

這個範例展示如何準備用於微調的訓練資料！

## 什麼是 Fine-Tuning（微調）？

Fine-Tuning 是在已經訓練好的模型上，用特定的資料再次訓練，讓模型學會特定領域的知識或風格。

```
基礎模型（通用知識）
      │
      ▼ Fine-Tuning
微調模型（特定領域專家）
```

## 學習目標
- 了解 Fine-Tuning 資料格式
- 學會建立訓練資料集
- 認識不同的資料格式

## Fine-Tuning 資料格式

通常是「對話」形式：
- **輸入**：問題或指令
- **輸出**：期望的回答

## Step 1: 匯入套件

In [ ]:
import json

## Step 2: 了解訓練資料格式

### 格式 1：指令格式（Instruction Format）

In [ ]:
def create_training_example(instruction, input_text, output):
    """
    建立一筆訓練資料（指令格式）

    參數：
        instruction: 任務指令
        input_text: 輸入內容（可為空）
        output: 期望的輸出

    回傳：
        格式化的訓練資料
    """
    if input_text:
        text = f"### 指令：\n{instruction}\n\n### 輸入：\n{input_text}\n\n### 回答：\n{output}"
    else:
        text = f"### 指令：\n{instruction}\n\n### 回答：\n{output}"

    return {"text": text}

In [ ]:
# 範例：指令格式
example1 = create_training_example(
    instruction="將以下英文翻譯成中文",
    input_text="Hello, how are you?",
    output="你好，你好嗎？"
)

print("指令格式範例：")
print(example1["text"])

### 格式 2：對話格式（Chat Format）

這是 OpenAI 使用的格式，也是目前最常見的格式。

In [ ]:
def create_chat_example(system, user, assistant):
    """
    建立對話格式的訓練資料（OpenAI 格式）

    參數：
        system: 系統提示詞（定義 AI 角色）
        user: 使用者訊息
        assistant: AI 回答
    """
    return {
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ]
    }

In [ ]:
# 範例：對話格式
example2 = create_chat_example(
    system="你是一個友善的翻譯助手",
    user="請把 'Hello, how are you?' 翻譯成中文",
    assistant="你好，你好嗎？"
)

print("對話格式範例：")
print(json.dumps(example2, ensure_ascii=False, indent=2))

## Step 3: 建立客服機器人訓練資料

In [ ]:
def create_customer_service_dataset():
    """
    建立客服機器人的訓練資料集
    """

    system_prompt = "你是一個友善的線上商店客服助手，專門回答關於訂單和產品的問題。"

    training_data = [
        create_chat_example(
            system_prompt,
            "我的訂單什麼時候會到？",
            "您好！一般訂單會在 3-5 個工作天內送達。如果您能提供訂單編號，我可以幫您查詢更準確的配送狀態。請問您的訂單編號是多少呢？"
        ),
        create_chat_example(
            system_prompt,
            "我想退貨",
            "沒問題！我們提供 7 天內無條件退貨服務。請確保商品保持原包裝且未使用。您可以透過以下步驟申請退貨：\n1. 登入會員中心\n2. 點選「我的訂單」\n3. 選擇要退貨的商品\n4. 填寫退貨原因\n\n請問需要我協助您其他事項嗎？"
        ),
        create_chat_example(
            system_prompt,
            "這個產品有保固嗎？",
            "是的，我們所有產品都提供原廠保固！保固期限依產品類型而定：\n- 電子產品：1 年保固\n- 家電產品：2 年保固\n- 配件類：6 個月保固\n\n請問您想了解哪個產品的保固資訊呢？"
        ),
        create_chat_example(
            system_prompt,
            "可以改寄送地址嗎？",
            "可以的！如果訂單尚未出貨，您可以修改寄送地址。請提供：\n1. 您的訂單編號\n2. 新的寄送地址\n\n我會盡快為您處理。不過如果訂單已經出貨，就需要聯繫物流公司進行轉寄囉。"
        ),
        create_chat_example(
            system_prompt,
            "付款失敗怎麼辦？",
            "別擔心！付款失敗可能有幾個原因：\n1. 信用卡額度不足\n2. 銀行拒絕交易\n3. 網路連線問題\n\n建議您可以：\n- 確認信用卡狀態正常\n- 嘗試使用其他付款方式\n- 稍後重新下單\n\n如果問題持續發生，請提供錯誤訊息，我可以進一步協助您！"
        ),
    ]

    return training_data

In [ ]:
# 建立資料集
dataset = create_customer_service_dataset()

print(f"建立了 {len(dataset)} 筆訓練資料\n")

# 預覽資料
print("=== 資料集預覽 ===")
for i, item in enumerate(dataset[:2], 1):
    print(f"\n--- 範例 {i} ---")
    for msg in item["messages"]:
        role = msg['role'].upper()
        content = msg['content'][:80] + "..." if len(msg['content']) > 80 else msg['content']
        print(f"{role}: {content}")

## Step 4: 儲存資料集

### JSONL 格式

JSONL（JSON Lines）是訓練資料的標準格式：
- 每一行是一個獨立的 JSON 物件
- 易於處理大量資料
- 可以逐行讀取，不需要一次載入全部

In [ ]:
def save_dataset(data, filename):
    """
    儲存資料集為 JSONL 格式
    （每一行是一個 JSON 物件）
    """
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"已儲存 {len(data)} 筆資料到 {filename}")

In [ ]:
# 儲存客服資料集
save_dataset(dataset, "customer_service_training.jsonl")

In [ ]:
# 查看儲存的檔案內容
print("=== JSONL 檔案內容 ===")
with open("customer_service_training.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f, 1):
        print(f"Line {i}: {line[:100]}...")

## Step 5: 建立其他領域的資料集

In [ ]:
# 程式教學資料集
def create_coding_tutor_dataset():
    system_prompt = "你是一位耐心的程式設計教師，專門教導初學者學習 Python。"
    
    return [
        create_chat_example(
            system_prompt,
            "什麼是變數？",
            "變數就像是一個有名字的盒子，用來存放資料。\n\n想像你有一個便當盒，你可以在盒子上貼標籤叫做『午餐』。在程式中：\n```python\nlunch = '炒飯'\n```\n\n這裡 `lunch` 就是變數名稱（標籤），`'炒飯'` 就是存放的值（盒子裡的東西）。\n\n你可以隨時改變盒子裡的東西：\n```python\nlunch = '便當'  # 現在變成便當了\n```"
        ),
        create_chat_example(
            system_prompt,
            "for 迴圈怎麼用？",
            "for 迴圈用來重複執行某段程式碼！\n\n想像你要跟班上 5 位同學打招呼：\n```python\n同學們 = ['小明', '小華', '小美', '小強', '小玲']\nfor 同學 in 同學們:\n    print(f'嗨，{同學}！')\n```\n\n輸出會是：\n```\n嗨，小明！\n嗨，小華！\n嗨，小美！\n嗨，小強！\n嗨，小玲！\n```\n\n程式會自動對每位同學執行一次打招呼！"
        )
    ]

coding_dataset = create_coding_tutor_dataset()
save_dataset(coding_dataset, "coding_tutor_training.jsonl")

## Step 6: 讀取和驗證資料集

In [ ]:
def load_dataset(filename):
    """
    載入 JSONL 格式的資料集
    """
    data = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    return data

def validate_dataset(data):
    """
    驗證資料集格式是否正確
    """
    issues = []
    
    for i, item in enumerate(data):
        if "messages" not in item:
            issues.append(f"第 {i+1} 筆：缺少 messages 欄位")
            continue
            
        messages = item["messages"]
        roles = [m.get("role") for m in messages]
        
        # 檢查是否有 assistant 回應
        if "assistant" not in roles:
            issues.append(f"第 {i+1} 筆：缺少 assistant 回應")
        
        # 檢查是否有 user 訊息
        if "user" not in roles:
            issues.append(f"第 {i+1} 筆：缺少 user 訊息")
    
    if issues:
        print("發現以下問題：")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print(f"✓ 資料集格式正確！共 {len(data)} 筆資料")
    
    return len(issues) == 0

In [ ]:
# 載入並驗證資料集
loaded_data = load_dataset("customer_service_training.jsonl")
validate_dataset(loaded_data)

## 資料集品質建議

### 好的訓練資料應該：

1. **多樣性**：涵蓋各種問題類型
2. **一致性**：回答風格統一
3. **完整性**：回答要完整、有幫助
4. **正確性**：資訊要準確

### 數量建議：

| 任務類型 | 建議數量 |
|----------|----------|
| 簡單任務 | 50-100 筆 |
| 中等任務 | 100-500 筆 |
| 複雜任務 | 500+ 筆 |

## 練習區

In [ ]:
# 建立你自己的訓練資料集
# 例如：餐廳訂位助手、學校問答機器人、健康諮詢助手等

def create_my_dataset():
    system_prompt = "你是..."
    
    return [
        create_chat_example(
            system_prompt,
            "問題 1",
            "回答 1"
        ),
        # 加入更多範例...
    ]

# 取消註解來建立你的資料集
# my_dataset = create_my_dataset()
# save_dataset(my_dataset, "my_training.jsonl")

## 重點回顧

1. **訓練資料格式**：
   - 指令格式：適合單輪任務
   - 對話格式：適合聊天應用

2. **JSONL 格式**：每行一個 JSON，方便處理大量資料

3. **資料品質**：多樣、一致、完整、正確

4. **驗證**：確保資料格式正確再開始訓練

## 下一步

在下一個範例中，我們將學習如何使用 Ollama 建立自訂模型！